In [21]:
from math import sqrt
import torch
from torch_geometric.data import Data
from random import randint
from sys import float_info

In [42]:
instances = {}
for k in range(0, 5000):
    nodes = {}
    for i in range(0, 50):
        lat_i = randint(0, 100)
        lon_i = randint(0, 100)
        node_i = (lat_i, lon_i)
        lat_j = randint(0, 100)
        lon_j = randint(0, 100)
        node_j = (lat_j, lon_j)
        nodes[i + 1] = node_i
        nodes[i + 51] = node_j

    dist = {}
    pairs = {}
    for i in range(1, 101):
        for j in range(1, 101):
            if i != j:
                dist[i,j] = sqrt( (nodes[i][0] - nodes[j][0])**2 + (nodes[i][1] - nodes[j][1])**2 )
            else:
                dist[i,j] = float_info.max
    for i in range(1, 101):
        for j in range(1, 101):
            if i not in pairs:
                pairs[i] = j
            if i != j:
                if dist[i,j] < dist[i,pairs[i]]:
                    pairs[i] = j

    nodes[0] = (0,0)
    for i in range(1,101):
        dist[0,i] = sqrt( (nodes[0][0] - nodes[i][0])**2 + (nodes[0][1] - nodes[i][1])**2 )
        dist[i,0] = dist[0,i]
    y = [[0 for _ in range(101)] for _ in range(101)]
    for i in range(101):
        for j in range(101):
            if i > 0:
                if dist[0,i] < dist[0,pairs[i]]:
                    y[pairs[i]][i] = 1
                else:
                    y[i][pairs[i]] = 1
                
    instances[k] = {"nodes": nodes, "dist": dist, "y": y}

In [46]:

import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

In [39]:
complete_graph_list = []
for i in range(101):
    for j in range(101):
        if i != j:
            complete_graph_list.append([i,j])
edge_index = torch.tensor(complete_graph_list, dtype=torch.double).t().contiguous()
n_edges = len(complete_graph_list)

In [44]:
data_list = []
for instance_name in instances:
    y = torch.tensor(instances[instance_name]["y"], dtype=torch.double)
    x = torch.tensor([instances[instance_name]["nodes"][i] for i in range(0, 101)], dtype=torch.double)
    attr = [[i] for i in range(n_edges)]
    cnt = -1
    for i in range(101):
        for j in range(101):
            if i != j:
                cnt += 1
                attr[cnt].append(instances[instance_name]["dist"][i,j])
    attr = torch.tensor(attr, dtype=torch.double)
    pos = []
    for i in range(101):
        pos.append(instances[instance_name]["nodes"][i])
    pos = torch.tensor(pos, dtype=torch.double)
    # ## filtering by TW, strict
    # complete_graph_list = []
    # for i in range(101):
    #     for j in range(101):
    #         if i!=j:
    #             try:
    #                 if instance_dict[instance_name][i][5] + instance_dict[instance_name][i][6] + loc_dict[i][j] < instance_dict[instance_name][i][5]:
    #                     complete_graph_list.append([i,j])
    #             except:
    #                 continue
    # edge_index = torch.tensor(complete_graph_list, dtype=torch.double).t().contiguous()
    ## end filtering
    data_list.append(Data(x=x, y=y, edge_index=edge_index, pos=pos, edge_attr=attr))

In [47]:
# data_source = Instances(data_list)
dataloader = DataLoader(data_list[0:1468], batch_size=1)
data_test = DataLoader(data_list[1468:1488], batch_size=1)
# datatorch = data_source.to_conv_nets(start=428, end=1458, batch_size=10)
# torchtest = data_source.to_conv_nets(start=1458, end=488, batch_size=10)

In [48]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing

class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='max') #  "Max" aggregation.
        self.mlp = Seq(Linear(2 * in_channels, out_channels),
                       ReLU(),
                       Linear(out_channels, out_channels))

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)

In [49]:
from torch_geometric.nn import knn_graph

class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=6):
        super().__init__(in_channels, out_channels)
        self.k = k
        self.double()

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super().forward(x, edge_index)

In [58]:
def myCustomLoss(my_outputs, objective_matrix):
    #specifying the batch size
    my_batch_size = my_outputs.size()[0]
    #selecting the values that correspond to labels
    diff = torch.abs(my_outputs - objective_matrix)
    return torch.sum(diff*diff)/(101*100)

In [59]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output = model(data.x)
        # print(output, data.y)
        # print(output)
        loss = myCustomLoss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
        test_size = 101*100*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x)
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
        test_size = 101*100*1 #extract batch size
    return correct / test_size

In [51]:
from copy import deepcopy

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DynamicEdgeConv(2, 101).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 31):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 0.0919, Train Acc: 0.1998, Test Acc: 0.0003
Epoch: 002, Loss: 0.0014, Train Acc: 1.5092, Test Acc: 0.0100
Epoch: 003, Loss: 0.0013, Train Acc: 2.3975, Test Acc: 0.0498
Epoch: 004, Loss: 0.0013, Train Acc: 0.2928, Test Acc: 0.0099
Epoch: 005, Loss: 0.0013, Train Acc: 0.1654, Test Acc: 0.0001
Epoch: 006, Loss: 0.0013, Train Acc: 0.0035, Test Acc: 0.0001
Epoch: 007, Loss: 0.0013, Train Acc: 0.0002, Test Acc: 0.0000
Epoch: 008, Loss: 0.0013, Train Acc: 0.0099, Test Acc: 0.0000
Epoch: 009, Loss: 0.0013, Train Acc: 0.0099, Test Acc: 0.0000
Epoch: 010, Loss: 0.0013, Train Acc: 0.0099, Test Acc: 0.0000
Epoch: 011, Loss: 0.0013, Train Acc: 0.0016, Test Acc: 0.0000
Epoch: 012, Loss: 0.0012, Train Acc: 0.0003, Test Acc: 0.0000
Epoch: 013, Loss: 0.0012, Train Acc: 0.0012, Test Acc: 0.0000
Epoch: 014, Loss: 0.0012, Train Acc: 0.0004, Test Acc: 0.0000
Epoch: 015, Loss: 0.0012, Train Acc: 0.0009, Test Acc: 0.0000
Epoch: 016, Loss: 0.0012, Train Acc: 0.0025, Test Acc: 0.0001
Epoch: 0